In [102]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [103]:
#The Encoder network for single view 3D reconstruction is a ResNet18 pretrained
#on the ImageNet dataset with the last fully-connected layer adjusted to project
#the features to a 256 dimensional embedding, "c"
from torchvision.models.resnet import resnet18 as _resnet18


In [104]:
class ImageEncoder(nn.Module):
    def __init__(self):
        super(ImageEncoder,self).__init__()
        self.encoderModel = _resnet18(pretrained=True)
        self.fc1 = nn.Linear(1000, 256)
        self.betafc = nn.Linear(256,256)
        self.gammafc = nn.Linear(256,256)
        
    def forward(self,x):
        x = self.encoderModel(x)
        #project to 256 dimensional embedding 
        x = self.fc1(x)
        # Obtain Beta and gamma inputs into conditional batch normalization
        # QUESTION Are these split or one after the other?
        beta = self.betafc(x)
        gamma = self.gammafc(x) #? gammaLayer(beta)
        return beta,gamma

In [105]:
class Block(nn.Module):
    def __init__(self):
        super(Block,self).__init__()
        self.fc1 = nn.Linear(256,256)
        self.fc2 = nn.Linear(256,256)
        self.bn1 = nn.BatchNorm1d(256)
        self.bn2 = nn.BatchNorm1d(256)

        
    def forward(self,y):
        x = y['ex']
        gamma = y['gamma']
        beta = y['beta']
        #First apply Conditional Batch Normalization
        out = gamma*self.bn1(x) + beta
        #Then ReLU activation function
        out = F.relu(out)
        #fully connected layer
        out = self.fc1(out)
        #Second CBN layer
        out = gamma*self.bn2(out) + beta
        #RELU activation
        out = F.relu(out)
        #2nd fully connected
        out = self.fc2(out)
        #Add to the input of the ResNet Block 
        out = x + out
        
        return {'ex':out, 'beta':beta, 'gamma':gamma}

In [106]:
class OccupancyModel(nn.Module):
    def __init__(self):
        super(OccupancyModel,self).__init__()
        self.blocks = self.makeBlocks()
        self.encoder = ImageEncoder()
        self.cbn = nn.BatchNorm1d(256)
        self.fc1 = nn.Linear(3,256)
        self.fc2 = nn.Linear(256,1)
        
    def makeBlocks(self):
        blocks = []
        for _ in range(5):
            blocks.append(Block())
        return nn.Sequential(*blocks)
   
  
    def forward(self,x, img):
        gamma,beta = self.encoder(img)
        x = self.fc1(x)
        #5 pre-activation ResNet-blocks
        x = self.blocks({'gamma':gamma, 'beta':beta, 'ex':x })
        x = x['ex']
        x = gamma*self.cbn(x) + beta
        x = F.relu(x)
        x = self.fc2(x)
        x = torch.sigmoid(x)
        return x

In [139]:
model = OccupancyModel()
#Input to the occupancy network architecture is the 
#output of the encoder network and a batch of 3D coordinates. 
coords = torch.rand(64,3)
image = torch.rand(64,3,7,7)
model.eval()

p = model(coords,image)


In [156]:
#load some data:
#The .npz contains "points, occupancies, loc, scale" 
import numpy
with numpy.load("/home/andrea/Documents/GradSchool/OccupancyNetworks/occupancy_networks/data/ShapeNet/02691156/fd528602cbde6f11bbf3143b1cb6076a/points.npz") as data:
    pts = torch.tensor(data["points"], dtype=torch.float)
    occupancies = torch.tensor(numpy.unpackbits(data["occupancies"])[:pts.size()[0]], dtype=torch.float)

from PIL import Image
image = numpy.array(Image.open("/home/andrea/Documents/GradSchool/OccupancyNetworks/occupancy_networks/data/ShapeNet/02691156/fd528602cbde6f11bbf3143b1cb6076a/img_choy2016/015.jpg"))
#At least for this image directory, the jpgs come in as 137,137,3
image = torch.tensor(image,dtype=torch.float).permute(2,0,1)
image = image.view(1,3,137,137)

train_loader = torch.utils.data.DataLoader(list(zip(pts,occupancies)), batch_size=64)


In [161]:
def train(epoch, model, trainloader, optimizer):
    modelCriterion = nn.BCELoss()
    model.train()
    for batch_idx, data in enumerate(train_loader):
        (pts, occupancies) = data
        optimizer.zero_grad()
        output = model(pts, image) #a probability for each point 
        loss = modelCriterion(output, occupancies)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader),
                loss.item()))

In [162]:
optimizer = optim.Adam(model.parameters(), lr = 0.0001)
train(1,model,train_loader,optimizer)

Train Epoch: 1 [0/100000 (0%)]\tLoss: 0.001071
Train Epoch: 1 [20/100000 (1%)]\tLoss: 0.000012
Train Epoch: 1 [40/100000 (1%)]\tLoss: 0.003240
Train Epoch: 1 [60/100000 (2%)]\tLoss: 0.009982
Train Epoch: 1 [80/100000 (3%)]\tLoss: 0.064105
Train Epoch: 1 [100/100000 (3%)]\tLoss: 0.040212
Train Epoch: 1 [120/100000 (4%)]\tLoss: 0.003784
Train Epoch: 1 [140/100000 (4%)]\tLoss: 0.092932
Train Epoch: 1 [160/100000 (5%)]\tLoss: 0.008418
Train Epoch: 1 [180/100000 (6%)]\tLoss: 0.008762
Train Epoch: 1 [200/100000 (6%)]\tLoss: 0.048621
Train Epoch: 1 [220/100000 (7%)]\tLoss: 0.004228
Train Epoch: 1 [240/100000 (8%)]\tLoss: 0.006790
Train Epoch: 1 [260/100000 (8%)]\tLoss: 0.004797
Train Epoch: 1 [280/100000 (9%)]\tLoss: 0.041292
Train Epoch: 1 [300/100000 (10%)]\tLoss: 0.067860
Train Epoch: 1 [320/100000 (10%)]\tLoss: 0.004231
Train Epoch: 1 [340/100000 (11%)]\tLoss: 0.002475
Train Epoch: 1 [360/100000 (12%)]\tLoss: 0.107249
Train Epoch: 1 [380/100000 (12%)]\tLoss: 0.012398
Train Epoch: 1 [400/1

/home/andrea/.local/lib/python3.8/site-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
